In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout,BatchNormalization,regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess

 # **Model Configration**

In [ ]:
model_configs = {
    "vgg16": {
        "model_class": VGG16,
        "preprocess_fn": vgg16_preprocess,
        "input_shape": (224, 224, 3)
    },
    "vgg19": {
        "model_class": VGG19,
        "preprocess_fn": vgg19_preprocess,
        "input_shape": (224, 224, 3)
    },
    "inceptionv3": {
        "model_class": InceptionV3,
        "preprocess_fn": inception_preprocess,
        "input_shape": (299, 299, 3)  # Inception requires larger input
    }
}


NameError: name 'VGG16' is not defined

  # **Prepart the data for the models**

In [ ]:
def setup_datagenerator(train_dir, val_dir, test_dir, preprocess_fn, batch_size=16, image_size=(224, 224)):
    """
    Create and configure data generators for the model.
    
    Parameters:
    - train_dir: Path to training data directory
    - val_dir: Path to validation data directory
    - test_dir: Path to test data directory
    - preprocess_fn: Preprocessing function to apply to images
    - batch_size: Batch size for generators (default: 32)
    - image_size: Target image size as (height, width) (default: (224, 224))
    
    Returns:
    - train_generator: Training data generator
    - validation_generator: Validation data generator
    - test_generator: Test data generator
    """
    # Create data generators with appropriate preprocessing
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        preprocessing_function=preprocess_fn
    )
    
    # Validation and test generators don't need rescaling if preprocess_fn handles it
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_fn)
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_fn)
    
    # Create generators
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True
    )
    
    validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False
    )
    
    return train_generator, validation_generator, test_generator

  # **Build the model**

In [ ]:
def build_model(base_model_class, preprocess_fn, num_classes, input_shape=(224, 224, 3)):
    """Create a transfer learning model with the given base model"""
    # Create the base model
    base_model = base_model_class(
        include_top=False, 
        weights='imagenet',
        input_shape=input_shape
    )
    
    # Freeze the base model
    """base_model.trainable = False
    
    # Create new model on top
    inputs = tf.keras.Input(shape=input_shape)
    x = preprocess_fn(inputs)
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs, outputs)"""
    
     # Freeze the base model
    base_model.trainable = False
    

    inputs = Input(shape=input_shape)
    x = preprocess_fn(inputs)
    x = base_model(x, training=False) # Keep False to freeze base model initially

# Feature extraction and regularization head
    x = GlobalAveragePooling2D()(x)

# Add BatchNormalization to stabilize and accelerate training
    x = BatchNormalization()(x)

# Use a smaller, regularized Dense layer
    x = Dense(256, activation='relu', # 256 is often sufficient
                 kernel_regularizer=regularizers.l2(1e-4))(x) # Small L2 penalty
    x = Dropout(0.5)(x)

# Optional: Second smaller Dense layer for more capacity
    x = Dense(128, activation='relu',
                 kernel_regularizer=regularizers.l2(1e-4))(x)
    x = Dropout(0.3)(x) # Slightly lower dropout

# Final output layer
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [13]:
train_dir, val_dir, test_dir = r"F:\Rough preprocess image\train",r"F:\Rough preprocess image\validation",r"F:\Rough preprocess image\test"
train_gen,val_gen, test_gen =setup_datagenerator(train_dir, val_dir, test_dir,preprocess_input)

Found 7749 images belonging to 4 classes.
Found 861 images belonging to 4 classes.
Found 1182 images belonging to 4 classes.


In [39]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

# Define parameters
input_shape = (224, 224, 3)
num_classes = 4  # change to match your dataset


In [40]:

# Load VGG16 base (without top layers)
base_model = VGG16

# Build model
model = build_model(base_model_class=base_model, preprocess_fn=preprocess_input,input_shape= input_shape,num_classes= num_classes)

# Check architecture
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_3          │ (None, 224, 224)  │          0 │ input_layer_4[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_4          │ (None, 224, 224)  │          0 │ input_layer_4[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_5          │ (None, 224, 224)  │          0 │ input_layer_4[0]… │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack_1 (Stack)     │ (None, 224, 224,  │          0 │ get_item_3[0][0], │
│                     │ 3)                │            │ get_item_4[0][0], │
│                     │                   │            │ get_item_5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 224, 224,  │          0 │ stack_1[0][0]     │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 7, 7, 512) │ 14,714,688 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1024)      │    525,312 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 1024)      │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 4)         │      4,100 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 15,244,100 (58.15 MB)

 Trainable params: 529,412 (2.02 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
def train_model(model_name, config, train_gen, val_gen, train_dir, validation_dir, test_dir, 
                batch_size, num_classes, epochs=50):
    """Train the specified model and return history and trained model"""
    print(f"\nTraining model {model_name}.....")
    
    # Set up model-specific generators if input size differs
    if config['input_shape'] != (224, 224, 3):
        print(f"Using custom input shape: {config['input_shape']}")
        custom_train_gen, custom_val_gen, _ = setup_datagenerator(
            train_dir, validation_dir, test_dir,
            config['preprocess_fn'], batch_size, config['input_shape'][:2]
        )
        train_data = custom_train_gen
        val_data = custom_val_gen
    else:
        # Use the default generators
        train_data = train_gen
        val_data = val_gen
    
    # Build the model
    model = build_model(
        config['model_class'],
        config['preprocess_fn'],
        num_classes,
        config['input_shape']
    )
    
    # Set up callbacks
    checkpoint = ModelCheckpoint(
        f'{model_name}_best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    )
    
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    )
    
    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
    
    tensorboard = TensorBoard(log_dir=f'logs/{model_name}')
    
    # Train the model
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=[checkpoint, early_stopping, reduce_lr, tensorboard],
        verbose=1
    )
    
    return model, history

In [48]:
# Loop over all models in model_configs and train them
for model_name, config in model_configs.items():
    print(f"\n================= Training {model_name.upper()} =================\n")
    
    # Call train_model for each architecture
    model, history = train_model(
        model_name=model_name,
        config=config,
        train_gen=train_gen,
        val_gen=val_gen,
        train_dir="processed_dataset/train",
        validation_dir="processed_dataset/validation",
        test_dir=test_gen,
        batch_size=32,
        num_classes=num_classes,  # e.g., 5 classes
        epochs=5
    )
    
    # Save each final model after training
    model.save(f"results/{model_name}_final_model.keras")
    print(f"✅ Finished training {model_name}, model saved to results/{model_name}_final_model.keras\n")



================= Training VGG16 =================


Training model vgg16.....
Epoch 1/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 0s 22s/step - accuracy: 0.2954 - loss: 1.5001 

c:\Users\PMYLS\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()



Epoch 1: val_accuracy improved from -inf to 0.13705, saving model to vgg16_best_model.keras
485/485 ━━━━━━━━━━━━━━━━━━━━ 10936s 23s/step - accuracy: 0.2955 - loss: 1.4999 - val_accuracy: 0.1370 - val_loss: 40.8084 - learning_rate: 0.0010
Epoch 2/5
485/485 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3406 - loss: 1.2847

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/functional_3_1/vgg16_1/block4_conv2_1/Relu defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a memory object, in file tensorflow/core/kernels/mkl/mkl_conv_ops.cc:1112
	 [[{{node StatefulPartitionedCall/functional_3_1/vgg16_1/block4_conv2_1/Relu}}]] [Op:__inference_multi_step_on_iterator_18021]